## 1. Importar Librerías

In [ ]:
import asyncio
import random
from typing import Annotated
from datetime import datetime
from agent_framework import ChatAgent
from agent_framework.openai import OpenAIChatClient
from azure.identity import DefaultAzureCredential
from azure.identity.aio import get_bearer_token_provider
from pydantic import Field
from dotenv import load_dotenv
import os

## 2. Cargar Configuración

In [ ]:
load_dotenv()

print(f"Endpoint: {os.getenv('AZURE_OPENAI_ENDPOINT')}")
print(f"Modelo: {os.getenv('MODEL')}")

## 3. Configurar Cliente

In [ ]:
client = OpenAIChatClient(
    base_url=os.getenv("AZURE_OPENAI_ENDPOINT") + "/openai/v1/",
    api_key=get_bearer_token_provider(
        DefaultAzureCredential(), 
        "https://cognitiveservices.azure.com/.default"
    ),
    model_id=os.getenv("MODEL")
)

## 4. Definir Herramienta: get_weather

Simula la consulta del clima para una ciudad en una fecha específica.

In [ ]:
def get_weather(
    city: Annotated[str, Field(description="City name, spelled out fully")],
) -> dict:
    """Obtiene el clima para una ciudad."""
    print(f"🌤️  Getting weather for {city}")
    
    if random.random() < 0.5:
        return {"temperature": 72, "description": "Sunny"}
    else:
        return {"temperature": 60, "description": "Rainy"}

## 5. Definir Herramienta: get_activities

Retorna una lista de actividades disponibles en una ciudad para una fecha específica.

In [ ]:
def get_activities(
    city: Annotated[str, Field(description="The city to get activities for.")],
    date: Annotated[str, Field(description="The date to get activities for in format YYYY-MM-DD.")],
) -> list[dict]:
    """Obtiene actividades disponibles para una ciudad en una fecha."""
    print(f"🎯 Getting activities for {city} on {date}")
    
    return [
        {"name": "Hiking", "location": city},
        {"name": "Beach", "location": city},
        {"name": "Museum", "location": city},
    ]

## 6. Definir Herramienta: get_current_date

Retorna la fecha actual en formato YYYY-MM-DD.

In [ ]:
def get_current_date() -> str:
    """Obtiene la fecha actual del sistema."""
    print("📅 Getting current date")
    return datetime.now().strftime("%Y-%m-%d")

# Probar la función
print(f"Fecha actual: {get_current_date()}")

## 7. Crear el Agente con Todas las Herramientas

Este agente puede usar las tres herramientas para planificar fines de semana. Las instrucciones le indican:
- Considerar el clima al recomendar actividades
- Incluir la fecha en la respuesta
- No sugerir actividades inadecuadas para el clima

In [ ]:
agent = ChatAgent(
    chat_client=client, 
    instructions=(
        "You help users plan their weekends and choose the best activities for the given weather. "
        "If an activity would be unpleasant in weather, don't suggest it. "
        "Include date of the weekend in response."
    ), 
    tools=[get_weather, get_activities, get_current_date]
)

print("✅ Agente creado con 3 herramientas")

## 8. Ejecutar una Consulta Compleja

El agente automáticamente:
1. Llamará a `get_current_date()` para saber qué día es
2. Llamará a `get_weather()` para San Francisco
3. Llamará a `get_activities()` para obtener opciones
4. Combinará toda la información en una recomendación coherente

In [ ]:
async def main():
    print("\n" + "="*80)
    print("PLANIFICACIÓN DE FIN DE SEMANA")
    print("="*80 + "\n")
    
    response = await agent.run("hii what can I do this weekend in San Francisco?")
    
    print("\n" + "="*80)
    print("RESPUESTA DEL AGENTE")
    print("="*80 + "\n")
    print(response)

await main()

## 9. Probar con Diferentes Ciudades

Veamos cómo el agente adapta sus recomendaciones según el clima simulado.

In [ ]:
async def test_multiple_cities():
    cities = ["Miami", "Seattle", "New York"]
    
    for city in cities:
        print("\n" + "="*80)
        print(f"PLANIFICACIÓN PARA: {city.upper()}")
        print("="*80 + "\n")
        
        response = await agent.run(f"What activities would you recommend for this weekend in {city}?")
        print(response.text)
        print()

await test_multiple_cities()

## Conclusión

Este ejemplo avanzado demuestra:

### 1. **Múltiples Herramientas**
- El agente puede elegir entre varias funciones
- Puede llamar múltiples funciones en secuencia
- Integra resultados de diferentes fuentes

### 2. **Razonamiento Contextual**
- Considera el clima antes de recomendar actividades
- Usa la fecha actual para contexto temporal
- Filtra recomendaciones inadecuadas

### 3. **Orchestration Automática**
- El framework decide qué funciones llamar
- Determina el orden de las llamadas
- Maneja la composición de resultados

### Aplicaciones Prácticas:
- Asistentes de viaje
- Planificadores de eventos
- Sistemas de recomendación
- Agentes de atención al cliente con acceso a múltiples sistemas

El poder de este enfoque es que puedes agregar más herramientas sin cambiar la lógica del agente - solo necesitas definir nuevas funciones con descripciones claras.